In [1]:
import numpy as np
import gym
import random

In [2]:
# RANDOM AGENT
env = gym.make('Taxi-v2')
state = env.reset()
env.render()

total_reward = 0
done = False
while not done:
    state, reward, done, info = env.step(env.action_space.sample())
    total_reward += reward
    env.render()

print('Total reward:', total_reward)

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B

In [3]:
episodes = 1000
rewards = []
max_steps = 99

for episode in range(episodes):
    state = env.reset()  # Assuming you already have env created as above
    total_rewards = 0
    
    for step in range(max_steps):
        action = env.action_space.sample()  # TODO your policy here!
        state, reward, done, info = env.step(env.action_space.sample())
        total_rewards += reward
        if done:
            break
    rewards.append(total_rewards)        

print('Average score over time:', sum(rewards) / episodes)


Average score over time: -389.292


GOAL 1 - BEAT RANDOM 

In [4]:
action_size = env.action_space.n
print("Action size ", action_size)

state_size = env.observation_space.n
print("State size ", state_size)



Action size  6
State size  500


In [5]:
qtable = np.zeros((state_size, action_size))
print(qtable)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


In [6]:
total_episodes = 50000        # Total episodes
total_test_episodes = 100     # Total test episodes
max_steps = 99                # Max steps per episode

learning_rate = 0.7           # Learning rate
gamma = 0.618                 # Discounting rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability 
decay_rate = 0.01             # Exponential decay rate for exploration prob

In [7]:
# 2 For life or until learning is stopped
for episode in range(total_episodes):
    # Reset the environment
    state = env.reset()
    step = 0
    done = False
    
    for step in range(max_steps):
        # 3. Choose an action a in the current world state (s)
        ## First we randomize a number
        exp_exp_tradeoff = random.uniform(0,1)
        
        ## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
        if exp_exp_tradeoff > epsilon:
            action = np.argmax(qtable[state,:])
        
        # Else doing a random choice --> exploration
        else:
            action = env.action_space.sample()
        
        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done, info = env.step(action)

        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        qtable[state, action] = qtable[state, action] + learning_rate * (reward + gamma * 
                                    np.max(qtable[new_state, :]) - qtable[state, action])
                
        # Our new state is state
        state = new_state
        
        # If done : finish episode
        if done == True: 
            break
        
    episode += 1
    
    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)

In [8]:
env.reset()
rewards = []

for episode in range(total_test_episodes):
    state = env.reset()
    step = 0
    done = False
    total_rewards = 0
    print("****************************************************")
    print("EPISODE ", episode)

    for step in range(max_steps):
        env.render()
        # Take the action (index) that have the maximum expected future reward given that state
        action = np.argmax(qtable[state,:])
        
        new_state, reward, done, info = env.step(action)
        
        total_rewards += reward
        
        if done:
            rewards.append(total_rewards)
            print ("Score", total_rewards)
            break
        state = new_state
env.close()
print ("Score over time: " +  str(sum(rewards)/total_test_episodes))

****************************************************
EPISODE  0
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : |_: |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : :_: |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : :_: : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+-----

Score 8
****************************************************
EPISODE  70
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R: | : :G|
|_: : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| :_: : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : :_: : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: |_: :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | :_:G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
Score 12
****************************************************
EPISODE  71
+---------+
|R: | :

Score: 8.51

In [9]:
print(qtable)

[[  0.           0.           0.           0.           0.
    0.        ]
 [ -1.89497563  -1.45032808  -1.89503742  -1.45552788  -0.72512948
  -10.44824526]
 [ -0.72842139   0.44290025  -0.72618243   0.44476498   2.33782249
   -8.5514465 ]
 ...
 [ -2.06537874  -2.0024276   -2.12586921   0.44300716 -10.312666
  -11.02557912]
 [ -2.09014961  -1.44830235  -2.10916496  -2.08821364 -10.88307041
   -9.493666  ]
 [ 18.36411307  10.35535173  18.37681516  31.35602094   9.37596997
    9.37596997]]


GOAL 2:

Summary:
Nothing fancy just tuning the parameters.
-0.1 on the gamma seems to produce better result overall

Long Story Short:
I am still learning the basic of python apparently.  I initially tried to tune it by hand and got inconsistent result.  Then I moved in a batch mode approach and still got inconsistent result after what I thought were initialization problems.













In [88]:
#Reset to default parameters
total_episodes = 50000        # Total episodes
total_test_episodes = 100     # Total test episodes
max_steps = 99                # Max steps per episode

learning_rate = 0.7           # Learning rate
gamma = 0.618                 # Discounting rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability 
decay_rate = 0.01             # Exponential decay rate for exploration prob

In [111]:
def qlearn(total_episodes, learning_rate, gamma):
    qtable = np.zeros((state_size, action_size))
    rewards = []
#    total_episodes = 50000        # Total episodes
    max_steps = 99                # Max steps per episode

#    learning_rate = 0.7           # Learning rate
#    gamma = 0.618                 # Discounting rate

    # Exploration parameters
    epsilon = 1.0                 # Exploration rate
    max_epsilon = 1.0             # Exploration probability at start
    min_epsilon = 0.01            # Minimum exploration probability 
    decay_rate = 0.01             # Exponential decay rate for exploration prob
    # 2 For life or until learning is stopped
    for episode in range(total_episodes):
        # Reset the environment
        state = env.reset()
        step = 0
        done = False

        for step in range(max_steps):
            # 3. Choose an action a in the current world state (s)
            ## First we randomize a number
            exp_exp_tradeoff = random.uniform(0,1)

            ## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
            if exp_exp_tradeoff > epsilon:
                action = np.argmax(qtable[state,:])

            # Else doing a random choice --> exploration
            else:
                action = env.action_space.sample()

            # Take the action (a) and observe the outcome state(s') and reward (r)
            new_state, reward, done, info = env.step(action)

            # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
            qtable[state, action] = qtable[state, action] + learning_rate * (reward + gamma * 
                                        np.max(qtable[new_state, :]) - qtable[state, action])

            # Our new state is state
            state = new_state

            # If done : finish episode
            if done == True: 
                break

        episode += 1

        # Reduce epsilon (because we need less and less exploration)
        epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)
        
    return qtable


In [112]:
def score(qtable):
    env.reset()
    rewards = []
    total_test_episodes = 100     # Total test episodes

    for episode in range(total_test_episodes):
        state = env.reset()
        step = 0
        done = False
        total_rewards = 0
#        print("****************************************************")
#        print("EPISODE ", episode)

        for step in range(max_steps):
#            env.render()
            # Take the action (index) that have the maximum expected future reward given that state
            action = np.argmax(qtable[state,:])

            new_state, reward, done, info = env.step(action)

            total_rewards += reward

            if done:
                rewards.append(total_rewards)
#                print ("Score", total_rewards)
                break
            state = new_state
    env.close()
    print ("Score over time: " +  str(sum(rewards)/total_test_episodes))

In [104]:
hyperparameters = [
    [50000, 0.618, 0.7],
    [50000, 0.518, 0.7],
    [50000, 0.718, 0.7],
    [50000, 0.618, 0.6],
    [50000, 0.618, 0.7],
    [50000, 0.618, 0.8],
]
for params in hyperparameters:
    qlearn(params[0],params[1],params[2] )
    print(params)
    score()


[50000, 0.618, 0.7]
Score over time: 8.4
[50000, 0.518, 0.7]
Score over time: 8.61
[50000, 0.718, 0.7]
Score over time: 8.38
[50000, 0.618, 0.6]
Score over time: 8.33
[50000, 0.618, 0.7]
Score over time: 9.02
[50000, 0.618, 0.8]
Score over time: 8.58


-0.1 on the gamma seems to produce better result overall


OLD RESULTS BELOW - CAN IGNORE

NO GOAL 3 / Stretch Goal as usual.

# manually copy and pasted results....probably should not do this

# result 1

[50000, 0.618, 0.7]
Score over time: 8.26
[50000, 0.518, 0.7]
Score over time: 8.35
[50000, 0.718, 0.7]
Score over time: 8.15
[50000, 0.618, 0.6]
Score over time: 8.53
[50000, 0.618, 0.7]
Score over time: 8.42
[50000, 0.618, 0.8]
Score over time: 8.03
    
#result 2  (repeat)
[50000, 0.618, 0.7]
Score over time: 8.48
[50000, 0.518, 0.7]
Score over time: 8.39
[50000, 0.718, 0.7]
Score over time: 8.2
[50000, 0.618, 0.6]
Score over time: 8.03
[50000, 0.618, 0.7]
Score over time: 8.17
[50000, 0.618, 0.8]
Score over time: 8.41
    
# result 3 (repeat)
[50000, 0.618, 0.7]
Score over time: 8.44
[50000, 0.518, 0.7]
Score over time: 8.74
[50000, 0.718, 0.7]
Score over time: 8.29
[50000, 0.618, 0.6]
Score over time: 8.44
[50000, 0.618, 0.7]
Score over time: 9.08
[50000, 0.618, 0.8]
Score over time: 8.63
    
# run 4 (repeat)
[50000, 0.618, 0.7]
Score over time: 8.4
[50000, 0.518, 0.7]
Score over time: 8.61
[50000, 0.718, 0.7]
Score over time: 8.38
[50000, 0.618, 0.6]
Score over time: 8.33
[50000, 0.618, 0.7]
Score over time: 9.02
[50000, 0.618, 0.8]
Score over time: 8.58

In [105]:
hyperparameters = [
    [50000, 0.618, 0.7],
    [50000, 0.518, 0.7],
    [50000, 0.718, 0.7],
    [50000, 0.618, 0.6],
    [50000, 0.618, 0.8],
    
    [75000, 0.618, 0.7],
    [75000, 0.518, 0.7],
    [75000, 0.718, 0.7],
    [75000, 0.618, 0.6],
    [75000, 0.618, 0.8],
]
for params in hyperparameters:
    qlearn(params[0],params[1],params[2] )
    print(params)
    score()

[50000, 0.618, 0.7]
Score over time: 8.36
[50000, 0.518, 0.7]
Score over time: 8.2
[50000, 0.718, 0.7]
Score over time: 8.68
[50000, 0.618, 0.6]
Score over time: 8.73
[50000, 0.618, 0.8]
Score over time: 8.3
[75000, 0.618, 0.7]
Score over time: 8.35
[75000, 0.518, 0.7]
Score over time: 8.32
[75000, 0.718, 0.7]
Score over time: 8.3
[75000, 0.618, 0.6]
Score over time: 8.37
[75000, 0.618, 0.8]
Score over time: 8.55


In [ ]:
Manual Hyperparameters Tunings:
total_episodes = 100000 (+50000), gamma = 0.718 (+.1), Score over time: 8.61
total_episodes = 100000, gamma = 0.718, learning_rate = 0.8 (+.1), Score over time:  8.44
total_episodes = 150000 (+100000), gamma = 0.718 (+.1), Score over time: 8.57
total_episodes = 100000 (+50000), gamma = 0.768 (+.15), Score over time: 8.67
total_episodes = 100000 (+50000), gamma = 0.818 (+.2), Score over time: 7.77
total_episodes = 100000 (+50000), gamma = 0.788 (+.17), Score over time: 8.62
total_episodes = 50000, gamma = 0.618, learning_rate = 0.6 (-.1), Score over time:  8.34
total_episodes = 100000, gamma = 0.618, learning_rate = 0.6 (-.1), Score over time:  8.82
total_episodes = 100000, gamma = 0.718(+.1), learning_rate = 0.6 (-.1), Score over time:  8.45
total_episodes = 100000, gamma = 0.618, learning_rate = 0.5 (-.2), Score over time:  8.5
total_episodes = 150000, gamma = 0.618, learning_rate = 0.6 (-.1), Score over time:  8.5
total_episodes = 100000, gamma = 0.618, learning_rate = 0.6 (-.1), Score over time:  8.93 (rerun)
total_episodes = 100000, gamma = 0.618, learning_rate = 0.6 (-.1), Score over time:  8.44 (rerun2 - what!?!?)
# reset qtable and reward
total_episodes = 100000, gamma = 0.618, learning_rate = 0.6 (-.1), Score over time:  8.35 (rerun3)
total_episodes = 100000, gamma = 0.618, learning_rate = 0.6 (-.1), Score over time:  8.28 (rerun4)
total_episodes = 50000, gamma = 0.618, learning_rate = 0.6 (-.1), Score over time:  8.24 
total_episodes = 50000, gamma = 0.718 (+.1), Score over time: 8.1
default, Score over time: 8.53 (hmmmmm)
total_episodes = 100000 , Score over time: 8.41
total_episodes = 100000 , learning_rate = 0.6 (-.1), Score over time: 8.47
    
Summary: inconsistent, moved to batch mode testing above
